In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [35]:
data = pd.read_csv("train_v9rqX0R.csv")

In [36]:
test = pd.read_csv("test_AbJTz2l.csv")

In [37]:
def preprocess(df):
    df['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)
    df['Outlet_Size'] = df['Outlet_Size'].fillna('No data')
    df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({
        'LF': 'Low Fat',
        'low fat': 'Low Fat',
        'reg': 'Regular'
    })
    return df

In [38]:
data = preprocess(data)
test = preprocess(test)

In [ ]:
data.head()

In [18]:
categorical_cols = ['Item_Fat_Content', 'Item_Type', 
                    'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

In [19]:
y = data['Item_Outlet_Sales']
X = data.drop(['Item_Identifier','Item_Outlet_Sales','Outlet_Identifier'], axis=1)

In [24]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)
y_pred_rf = rf_model.predict(X)
rmse_rf = np.sqrt(mean_squared_error(y, y_pred_rf))
print("Random Forest Training RMSE:", rmse_rf)

Random Forest Training RMSE: 423.57370837454243


In [25]:
drop_cols = ['Item_Identifier', 'Outlet_Identifier']
X_test = test.drop(columns=drop_cols)

In [26]:
test_preds = rf_model.predict(X_test)
submission = test[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = test_preds
submission.to_csv('submission_RF_final.csv', index=False)

In [23]:
model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=15)
model.fit(X_train, y_train)
train_preds = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
print(f"Training RMSE: {train_rmse}")

Training RMSE: 647.7610920362239
